In [1]:
import pandas as pd
from datetime import datetime, date, timedelta
import glob
import os

import netCDF4
import xarray as xr
import numpy as np


In [2]:
start_time = '2021-05-25 00:00:00' # Modifiable
start_day = pd.Timestamp(start_time).strftime('%Y-%m-%d')
today = date.today()
yesterday = today - timedelta(days = 1)
now = datetime.now()
#end_time = end_day = yesterday.strftime('%Y-%m-%d')
end_time = end_day = '2021-09-01'
directory = 'Z:\\wse\\test\\'

merge_range = pd.date_range(start=start_time, end=end_time, freq='d')

merge_list = []

for i in merge_range:
    merge_list.append('y' + i.strftime('%Y') + '_d' + i.strftime('%j'))

lookup = {}
for i in merge_list:
    esc_set = directory + '*' + glob.escape(i) + '.nc'
    lookup[i] = []
    if (glob.glob(esc_set)) not in lookup.values():
        lookup.update({i:(glob.glob(esc_set))})
        

In [12]:
for i in merge_list:
    ds = xr.open_mfdataset(lookup[str(i)], combine = 'by_coords')
    t = pd.to_datetime(pd.to_datetime(ds['time'].values[0]).strftime('%Y-%m-%d'))
    day = int(round(t.timestamp())*1000000000)
    output_name = f'wse_500m_{day}_combine.nc'
    ds.to_netcdf(directory + output_name)

OSError: no files to open